In [1]:
import pandas as pd
import numpy as np



### Get all the pillar names from the excel

In [2]:
names = pd.read_excel('../../UNDP Digital Assessment Data Framework Filename Matching V7.xlsx')

In [3]:
col_names = ['Indicator','check', 'Data Source','Index','Filename','Sub-Pillar']

In [4]:
names = names[col_names]

In [5]:
names.head()

,Indicator,check,Data Source,Index,Filename,Sub-Pillar
0,Countries,NaN,United Nations,False,Countries,NaN
1,"Database of Global Administrative Areas (GADM,...",NaN,GADM maps and data,False,NaN,NaN
2,High Resolution Population Density Maps + Demo...,NaN,Facebook,False,NaN,NaN
3,population density vs openstreetmap object den...,NaN,Kontur,False,NaN,NaN
4,Population Density,Infrastructure,World Bank,False,population_density,Connectivity Technology


In [6]:
# get all the files per pillar
data_stats = names.groupby('check').agg({'Filename':'count','Indicator':'count'})

In [7]:
data_stats

,Filename,Indicator
check,,
Business,16,25
Foundations,14,19
Government,9,15
Infrastructure,39,48
People,34,47
Regulation,5,8
Strategy,1,1


### Foundations

In [8]:
bnames = names[(names.check=='Foundations')&(~names.Filename.isna())]#&(names.Index==False)]

In [9]:
bnames.head(25)

,Indicator,check,Data Source,Index,Filename,Sub-Pillar
149,Digital payments penetration,Foundations,Portulans Institute,TRUE,digital_payments_penetration,Digital Payments
150,% of population with digital finance account -...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
151,Made or received digital payments in the past ...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
152,Made or received digital payments in the past ...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
155,Used a mobile phone or the internet to check a...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
156,Used a mobile phone or the internet to check a...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
157,\n% Making online transactions - Men,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
158,% Making online transactions - Women,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
159,\n% of population with digital finance account...,Foundations,World Bank,False,population_digital_financial_services,Digital Payments
164,% of population with ID (all),Foundations,World Bank,False,id4d_nid,Digital Legal Identity


In [10]:
# get list of names for all indicators
indicators = bnames.Indicator.unique()
subpillars = bnames['Sub-Pillar'].unique()

In [11]:
# get all file names
bfiles = bnames.Filename.unique()

In [12]:
bfiles

array(['digital_payments_penetration',
       'population_digital_financial_services', 'id4d_nid',
       'id4d_services', 'Egov_strategy', 'open_data_idx'], dtype=object)

In [13]:
subpillars

array(['Digital Payments', 'Digital Legal Identity', 'Data Exchange'],
      dtype=object)

In [14]:
# ls digital-readiness-assessment-main/processed/

In [15]:
##ict_goods and services not in process data

In [16]:
# formula for converting scale
def convert_rank(old_value, old_min=1, old_max=7, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

### 1. Digital payments penetration

In [17]:
indicators[0]
subpillars[0]

'Digital Payments'

In [18]:
# load data
indicator = indicators[0]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillar = subpillars[0]
print(subpillar)

Digital payments penetration
digital_payments_penetration
Digital Payments


In [19]:
df.head()

,RANK,COUNTRY/ECONOMY,VALUE,SCORE
0,1.0,Norway,0.85,100.00
1,2.0,Denmark,0.83,97.24
2,3.0,Finland,0.80,93.95
3,4.0,Sweden,0.80,93.08
4,5.0,Netherlands,0.76,89.01


In [20]:
# score looks like the one to use
df.describe()

,RANK,VALUE,SCORE
count,122.000000,122.000000,122.000000
mean,61.500000,0.315738,36.076967
std,35.362409,0.209012,24.867406
min,1.000000,0.010000,0.000000
25%,31.250000,0.150000,16.235000
50%,61.500000,0.270000,30.870000
75%,91.750000,0.440000,51.110000
max,122.000000,0.850000,100.000000


In [21]:
# df.Indicator.unique()

In [22]:
# create standard columns
df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['SCORE'] 
df['Sub-Pillar'] = subpillar
df['Year'] = 2017


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-100 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [23]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']].head(15)

In [24]:
# output scores to csv
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 2. % of population with digital finance account - registered


In [25]:
indicator = indicators[1]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[0]
subpillar = subpillars[0]
print(subpillar)

% of population with digital finance account - registered
population_digital_financial_services
Digital Payments


In [26]:
df.head(15)

,Year,ISO,Country Name,Region,Income Group,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,2011,AFG,Afghanistan,South Asia,Low income,9%,15%,15%,2%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AFG,Afghanistan,South Asia,Low income,10%,16%,15%,4%,4%,...,0%,0%,0%,0%,0%,0%,1%,0%,NaN,NaN
2,2017,AFG,Afghanistan,South Asia,Low income,15%,23%,25%,4%,7%,...,1%,0%,1%,0%,2%,0%,1%,1%,NaN,NaN
3,2011,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,39%,39%,46%,31%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,29%,36%,36%,12%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,28%,34%,36%,15%,23%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,38%,43%,44%,29%,34%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,40%,42%,53%,27%,38%,...,2%,6%,1%,1%,4%,0%,4%,2%,NaN,NaN
8,2011,ARB,Arab world,NaN,NaN,22%,30%,33%,11%,14%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,ARB,Arab world,NaN,NaN,30%,38%,42%,18%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df = df[(df.Year == 2017)]

# Must convert the data to float by removing the % sign
df['Account (% age 15+)'] = df['Account (% age 15+)'].str.replace('%','')
df['Account (% age 15+)'] = df['Account (% age 15+)'].astype(float)

In [28]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Account (% age 15+)'] 
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [29]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Afghanistan,2017,% of population with digital finance account -...,15.0,1.329670,True,Digital Payments
7,Albania,2017,% of population with digital finance account -...,40.0,2.703297,True,Digital Payments
10,Arab world,2017,% of population with digital finance account -...,37.0,2.538462,True,Digital Payments
13,United Arab Emirates,2017,% of population with digital finance account -...,88.0,5.340659,True,Digital Payments
16,Argentina,2017,% of population with digital finance account -...,49.0,3.197802,True,Digital Payments
...,...,...,...,...,...,...,...
479,World,2017,% of population with digital finance account -...,69.0,4.296703,True,Digital Payments
482,Kosovo,2017,% of population with digital finance account -...,52.0,3.362637,True,Digital Payments
487,South Africa,2017,% of population with digital finance account -...,69.0,4.296703,True,Digital Payments
490,Zambia,2017,% of population with digital finance account -...,46.0,3.032967,True,Digital Payments


In [30]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 3. % of population with digital finance account - active (90 days)

In [31]:
indicator = indicators[2]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Made or received digital payments in the past year (% age 15+) - all
population_digital_financial_services
Digital Payments


In [32]:
df.head()

,Year,ISO,Country Name,Region,Income Group,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,2011,AFG,Afghanistan,South Asia,Low income,9%,15%,15%,2%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AFG,Afghanistan,South Asia,Low income,10%,16%,15%,4%,4%,...,0%,0%,0%,0%,0%,0%,1%,0%,NaN,NaN
2,2017,AFG,Afghanistan,South Asia,Low income,15%,23%,25%,4%,7%,...,1%,0%,1%,0%,2%,0%,1%,1%,NaN,NaN
3,2011,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,39%,39%,46%,31%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,29%,36%,36%,12%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df = df[(df.Year == 2017)]

# Must convert the data to float by removing the % sign
df['Made or received digital payments in the past year (% age 15+)'] = df['Made or received digital payments in the past year (% age 15+)'].str.replace('%','')
df['Made or received digital payments in the past year (% age 15+)'] = df['Made or received digital payments in the past year (% age 15+)'].astype(float)

In [34]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Made or received digital payments in the past year (% age 15+)'] 
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [35]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Afghanistan,2017,Made or received digital payments in the past ...,11.0,1.217391,True,Digital Payments
7,Albania,2017,Made or received digital payments in the past ...,29.0,2.195652,True,Digital Payments
10,Arab world,2017,Made or received digital payments in the past ...,26.0,2.032609,True,Digital Payments
13,United Arab Emirates,2017,Made or received digital payments in the past ...,84.0,5.184783,True,Digital Payments
16,Argentina,2017,Made or received digital payments in the past ...,40.0,2.793478,True,Digital Payments
...,...,...,...,...,...,...,...
479,World,2017,Made or received digital payments in the past ...,52.0,3.445652,True,Digital Payments
482,Kosovo,2017,Made or received digital payments in the past ...,39.0,2.739130,True,Digital Payments
487,South Africa,2017,Made or received digital payments in the past ...,60.0,3.880435,True,Digital Payments
490,Zambia,2017,Made or received digital payments in the past ...,39.0,2.739130,True,Digital Payments


In [36]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator))

## 4. % of population with ID (Total)


In [37]:
indicator = indicators[3]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Made or received digital payments in the past year (% age 15+) - female
population_digital_financial_services
Digital Legal Identity


In [38]:
df.head(15)

,Year,ISO,Country Name,Region,Income Group,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,2011,AFG,Afghanistan,South Asia,Low income,9%,15%,15%,2%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AFG,Afghanistan,South Asia,Low income,10%,16%,15%,4%,4%,...,0%,0%,0%,0%,0%,0%,1%,0%,NaN,NaN
2,2017,AFG,Afghanistan,South Asia,Low income,15%,23%,25%,4%,7%,...,1%,0%,1%,0%,2%,0%,1%,1%,NaN,NaN
3,2011,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,39%,39%,46%,31%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,29%,36%,36%,12%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,28%,34%,36%,15%,23%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,38%,43%,44%,29%,34%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,40%,42%,53%,27%,38%,...,2%,6%,1%,1%,4%,0%,4%,2%,NaN,NaN
8,2011,ARB,Arab world,NaN,NaN,22%,30%,33%,11%,14%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,ARB,Arab world,NaN,NaN,30%,38%,42%,18%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
# Subtract the unregister proportion column by 100 and absolute the value
df['data_col'] = df['UP in % of Country Population'].sub(100).abs()
df ['Year'] = 2018
df['Country Name'] = df.iloc[:,1]
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

KeyError: 'UP in % of Country Population'

In [ ]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

In [ ]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

In [ ]:
## 5. % of population with ID (Female)

In [ ]:
indicator = indicators[4]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[1]
subpillar = subpillars[1]
print(subpillar)

In [ ]:
df.head(15)

In [ ]:
df['% of UP that is Female'] = df['% of UP that is Female'].replace('n/a',np.nan)

In [ ]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
# Subtract the unregister proportion column by 100 and absolute the value
df['data_col'] = df['% of UP that is Female'].sub(100).abs()
df ['Year'] = 2018
df['Country Name'] = df.iloc[:,1]
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [ ]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

In [ ]:
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 6. can ID be used for transactions



In [ ]:
indicator = indicators[5]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[1]
subpillar = subpillars[1]
print(subpillar)

In [ ]:
df.head(20)

In [ ]:
df['Digitized ID system']= df['Digitized ID system'].replace('-',np.nan)
df['Digitized ID system']= df['Digitized ID system'].astype(float)

In [ ]:
df.head(15)

In [ ]:
# create standard columns
df.rename(columns={'Economy':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Year'] = 2018
df['Indicator'] = indicator
df['data_col'] = df['Digitized ID system']
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [ ]:
df = df[['Country Name','Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

In [ ]:
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 7. Is personal data siloed


In [ ]:
indicator = indicators[6]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# Must limit the database to the first 206 rows, the remaining rows do not contain any useful information
df = df.iloc[0:205,:]

# Drop the superfluous rows by dropping na
df = df[df['#'].notna()]

# Must convert data in the DPL column into float
df['DPL'] = df['DPL'].astype(float, errors = 'ignore')

subpillars[1]
subpillar = subpillars[1]
print(subpillar)

In [ ]:
df.head(104)

In [ ]:
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df.iloc[:,205]
df['Country Name'] = df['Economy']
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [ ]:
df = df[['Country Name','Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

In [ ]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

## 8. Open data index

In [ ]:
bnames

In [ ]:
indicator = indicators[7]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

subpillars[2]
subpillar = subpillars[2]
print(subpillar)

In [ ]:
df.head(15)

In [ ]:
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['score'] 
df['Country Name'] = df['name']
df['Year'] = 2016
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [ ]:
df = df[['Country Name','Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df.head(15)

In [ ]:
# output scores
df.to_csv('../indicator_scores/foundations_{}_scores.csv'.format(indicator), index=False)

### Score Aggregating

In [ ]:
import os


In [ ]:
# get list of files in scores folder
scores = os.listdir('../indicator_scores/')
scores = [s for s in scores if s.startswith('foundations')]

In [ ]:
scores

In [ ]:
# create a dataframe that concatenates all these file into one table
df = pd.concat([pd.read_csv('../indicator_scores/{}'.format(s)) for s in scores])    

In [ ]:
df

In [ ]:
# Data cleaning
df['new_rank_score'] = df['new_rank_score'].fillna(0)
df.sort_values(by=['Country Name'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

In [ ]:
df.head(15)

In [ ]:
df.describe()

In [ ]:
# checking country names
sorted(df['Country Name'].unique().tolist())

In [ ]:
# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()
df['Country Name'] = df['Country Name'].str.strip('**')
df['Country Name'] = df['Country Name'].str.strip('*')

In [ ]:
# checking country names
sorted(df['Country Name'].unique().tolist())

In [ ]:
# average indicator scores per country
agg_df = df.groupby(['Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [ ]:
agg_df.columns = ['agg_score', 'count_source' ]

In [ ]:
max_number_sources = agg_df.describe()['count_source']['max']

In [ ]:
agg_df['agg_score_wt'] = agg_df['agg_score']*(agg_df['count_source']/max_number_sources)

In [ ]:
agg_df.sort_values(by='agg_score', ascending=False, inplace=True)

In [ ]:
agg_df.head(25)

In [ ]:
agg_df.to_csv('../pillar_scores/foundation_scores_v0.csv')

In [ ]:
### Score Aggregating by Subpillars

In [ ]:
df.insert(0,'Pillar','Foundations')
df

# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()
df['Country Name'] = df['Country Name'].str.strip('**')
df['Country Name'] = df['Country Name'].str.strip('*')

In [ ]:
sub_df = df.groupby(['Pillar','Sub-Pillar','Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [ ]:
sub_df.columns = ['agg_score', 'count_source' ]

In [ ]:
max_number_sources = sub_df.describe()['count_source']['max']

In [ ]:
sub_df['agg_score_wt'] = sub_df['agg_score']*(sub_df['count_source']/max_number_sources)

In [ ]:
sub_df.to_csv('../subpillar_score/foundations_scores_subpillar_v0.csv')